In [2]:
import os
os.getcwd()

'/Users/kjh/Documents/innocity'

In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

BASE = Path('/Users/kjh/Documents/innocity/data')  # 네 로컬 경로

file_recent = BASE / "행정구역_읍면동_별_5세별_주민등록인구_2011년__20251201171507.csv"
file_early  = BASE / "행정구역_읍면동_별_5세별_주민등록인구_20251201171832.csv"

# ---------------------------------------------------------
# 1) 파일 읽기
# ---------------------------------------------------------
df1 = pd.read_csv(file_recent, encoding="cp949")
df2 = pd.read_csv(file_early,  encoding="cp949")

# ---------------------------------------------------------
# 2) 최근 파일(df1) 정리: 헤더 행 제거 + 코드/이름 추출
# ---------------------------------------------------------
df1 = df1[df1["13999001 항목"] != "13999001 항목"].copy()

df1["code"] = df1["A 행정구역(동읍면)별"].str.extract(r"^(\d+)", expand=False)
df1["name"] = df1["A 행정구역(동읍면)별"].str.replace(r"^\d+\s*", "", regex=True)

# 10자리 코드(읍면동)만 사용
df1 = df1[df1["code"].str.len() == 10].copy()

# ---------------------------------------------------------
# 3) 과거 파일(df2) 정리: 헤더 행 제거 + T2 인구만
#    ★ 인덱스 리셋을 apply 전에!
# ---------------------------------------------------------
df2 = df2[df2["13999001 항목"] != "13999001 항목"].copy()
df2 = df2[df2["13999001 항목"] == "T2 인구"].copy()
df2 = df2.reset_index(drop=True)      # ★ 여기서 먼저 리셋

admin_cols = [c for c in df2.columns if c.startswith("A 행정구역(동읍면)별")]

def extract_code_and_name(row):
    for col in admin_cols[::-1]:  # 가장 깊은 수준부터
        s = str(row[col])
        m = re.match(r"\s*(\d{10})\s*(.*)", s)
        if m:
            return pd.Series({"code": m.group(1), "name": m.group(2)})
    return pd.Series({"code": None, "name": None})

df2_extra = df2.apply(extract_code_and_name, axis=1)
df2 = pd.concat([df2, df2_extra], axis=1)  # ★ 여기서는 reset_index 다시 안 함
df2 = df2[~df2["code"].isna()]             # 시도/시군구 소계 제거

# ---------------------------------------------------------
# 4) 인구값 파싱 함수
# ---------------------------------------------------------
def parse_pop(val):
    """'-', '0 계' 등은 NA, 나머지는 숫자로"""
    if isinstance(val, str):
        v = val.strip()
        if v in ["", "-", "0 계", "0계"]:
            return np.nan
        v = v.replace(",", "")
        try:
            return float(v)
        except:
            return np.nan
    try:
        return float(val)
    except:
        return np.nan

# ---------------------------------------------------------
# 5) df1: 2011–2024 → long
# ---------------------------------------------------------
year_cols1 = [c for c in df1.columns if c.startswith("Y")]

data1_long = df1.melt(
    id_vars=["code", "name"],
    value_vars=year_cols1,
    var_name="year_col",
    value_name="pop_raw"
)
data1_long["year"] = data1_long["year_col"].str.extract(r"Y(\d{4})")[0].astype(int)
data1_long["pop"]  = data1_long["pop_raw"].apply(parse_pop)

# ---------------------------------------------------------
# 6) df2: 1997–2010 → long
# ---------------------------------------------------------
year_cols2 = [c for c in df2.columns if c.startswith("Y")]

data2_long = df2.melt(
    id_vars=["code", "name"],
    value_vars=year_cols2,
    var_name="year_col",
    value_name="pop_raw"
)
data2_long["year"] = data2_long["year_col"].str.extract(r"Y(\d{4})")[0].astype(int)
data2_long["pop"]  = data2_long["pop_raw"].apply(parse_pop)

In [12]:
# 이름→코드 테이블 (참고용)
df1_units = df1[["code","name"]].drop_duplicates()
df2_units = df2[["code","name"]].drop_duplicates()
combined_units = pd.concat([df1_units, df2_units]).drop_duplicates()

# 행정구역 변경 매핑 (old_code -> new_code)
rename_map = {
    # 10) 사벌면 → 11) 사벌국면 (2020.1.1)
    "4725032000": "4725032500",

    # 12) 압량읍(신설) ← 13) 압량면(폐지) (2020.1.1)  *실제 코드는 반대로: old=면, new=읍
    "4729036000": "4729025600",

    # 23) 고령읍 → 24) 대가야읍 (2015.4.2)
    "4783025000": "4783025300",

    # 30) 호명면 → 27) 호명읍 (2024.2.1 승격)
    "4790036000": "4790025300",

    # 33) 서면 → 35) 금강송면 (2015.4.1, 울진군 서면만)
    "4793032000": "4793039000",

    # 34) 원남면 → 36) 매화면 (2015.4.1, 울진군 원남면만)
    "4793034000": "4793040000",

    # 28) 상리면 → 31) 효자면 (2016.2.1)
    "4790032000": "4790042000",

    # 29) 하리면 → 32) 은풍면 (2016.2.1)
    "4790033000": "4790043000",

    # 2) 양북면 → 3) 문무대왕면 (2021.4.1)
    "4713031000": "4713031500",

    # 7–9) 공단1동 + 공단2동 → 공단동 (2021.1.1 통합)
    "4719062100": "4719070000",   # 공단1동
    "4719062200": "4719070000",   # 공단2동

    # 25) 금수면 → 26) 금수강산면 (2024.8.1)
    "4784035000": "4784035500",
}

def canon_code(code: str) -> str:
    code = str(code)
    return rename_map.get(code, code)

data1_long["canon_code"] = data1_long["code"].astype(str).apply(canon_code)
data2_long["canon_code"] = data2_long["code"].astype(str).apply(canon_code)


In [13]:
# 2024년에 관측되는 이름을 canonical name으로 사용
latest_2024 = data1_long[data1_long["year"] == 2024].copy()
latest_2024 = latest_2024[latest_2024["canon_code"].str.len() == 10]

canon_name_map = latest_2024.set_index("canon_code")["name"].to_dict()

def canon_name(row):
    c = row["canon_code"]
    return canon_name_map.get(c, row["name"])

data1_long["canon_name"] = data1_long.apply(canon_name, axis=1)
data2_long["canon_name"] = data2_long.apply(canon_name, axis=1)

# 읍면동(10자리 코드)만 남기기
data1_long = data1_long[data1_long["canon_code"].str.len() == 10].copy()
data2_long = data2_long[data2_long["canon_code"].str.len() == 10].copy()


In [14]:
panel = pd.concat([data2_long, data1_long], ignore_index=True)

# 동일 canonical code·연도에 여러 행이 모일 수 있음(공단1/2동 → 공단동 같은 경우)
# → NA가 하나도 없으면 합계, 전부 NA면 NA
panel_agg = (
    panel.groupby(["canon_code", "canon_name", "year"])["pop"]
    .agg(lambda x: x.sum() if x.notna().any() else np.nan)
    .reset_index()
)

panel_agg["year"].min(), panel_agg["year"].max()
# (1997, 2024)


(np.int64(1997), np.int64(2024))

In [16]:
import pandas as pd
import numpy as np
from pathlib import Path

# 1) 2024년에 존재하는 읍면동 코드들만 대상으로 wide 변환
# codes_2024 = panel_agg.loc[panel_agg["year"] == 2024, "canon_code"].unique()

# sub = panel_agg[panel_agg["canon_code"].isin(codes_2024)].copy()

# 2) wide 형식으로 pivot (행: 읍면동, 열: 연도)
wide = (
    panel_agg.pivot_table(
        index=["canon_code", "canon_name"],
        columns="year",
        values="pop"
    )
    .reset_index()
)

# 3) 열 이름 정리: year → Y1997 식으로
year_cols = [c for c in wide.columns if isinstance(c, (int, np.integer))]
new_cols = {
    y: f"Y{y}" for y in year_cols
}
wide = wide.rename(columns=new_cols)

# 4) 열 순서 정리
ordered_cols = ["canon_code", "canon_name"] + sorted(
    [c for c in wide.columns if c.startswith("Y")]
)
wide = wide[ordered_cols]

# 5) CSV로 저장 (확인용)
wide.to_csv(BASE / "emd_2024_check_1997_cc.csv", index=False, encoding="euc-kr")

# 6) 상위 몇 행만 미리 보기
wide.head()


year,canon_code,canon_name,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,...,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023,Y2024
0,2772025000,군위읍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8069.0,7803.0
1,2772031000,소보면,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2317.0,2264.0
2,2772032000,효령면,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750.0,3657.0
3,2772033000,부계면,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2094.0,2076.0
4,2772034000,우보면,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1939.0,1917.0


### 행정구역 변동 적용

In [26]:
import pandas as pd
import numpy as np

def merge_administrative_districts(file_path, output_path):
    try:
        df = pd.read_csv(file_path, encoding='cp949')
    except:
        try:
            df = pd.read_csv(file_path, encoding='euc-kr')
        except:
            df = pd.read_csv(file_path) # Default
    
    df['canon_name'] = df['canon_name'].str.strip()

    # ---------------------------------------------------------
    # 1. 포항시 1997년 데이터 연결
    # ---------------------------------------------------------
    pohang_rows = df[df['canon_code'].astype(str).str.startswith('4711')]
    src_indices = pohang_rows[pohang_rows['Y1997'].notnull() & pohang_rows['Y1998'].isnull()].index.tolist()
    
    rows_to_drop = []
    for idx in src_indices:
        if idx not in df.index: continue
        row = df.loc[idx]
        name = row['canon_name']
        val_1997 = row['Y1997']
        candidates = df[(df['canon_code'].astype(str).str.startswith('4711')) & (df['canon_name'] == name) & (df.index != idx)]
        target_with_1998 = candidates[candidates['Y1998'].notnull()]
        target_idx = None
        if not target_with_1998.empty: target_idx = target_with_1998.index[0]
        elif not candidates.empty: target_idx = candidates.index[0]
        if target_idx is not None:
            df.loc[target_idx, 'Y1997'] = val_1997
            rows_to_drop.append(idx)
    df.drop(rows_to_drop, inplace=True)
    rows_to_drop = [] 

    # ---------------------------------------------------------
    # 1.5. 군위군 데이터 연결 (경북 4772 -> 대구 2772)
    # 2023년 7월 군위군 대구 편입으로 분리된 시계열 연결
    # ---------------------------------------------------------
    daegu_gunwi_mask = df['canon_code'].astype(str).str.startswith('2772')
    daegu_gunwi_rows = df[daegu_gunwi_mask]
    
    rows_to_drop_gunwi = []
    
    for idx, row in daegu_gunwi_rows.iterrows():
        name = row['canon_name']
        target_names = [name]
        
        # 고로면 -> 삼국유사면 변경 이력 처리
        if name == '삼국유사면':
            target_names.append('고로면')
        
        # 경북 군위군(4772)에서 해당 지역 찾기
        sources = df[
            (df['canon_code'].astype(str).str.startswith('4772')) & 
            (df['canon_name'].isin(target_names))
        ]
        
        if not sources.empty:
            cols = [c for c in df.columns if c.startswith('Y')]
            for s_idx, s_row in sources.iterrows():
                # 기간이 겹치지 않으므로(경북:~2022, 대구:2023~) 합산하여 연결
                df.loc[idx, cols] = df.loc[idx, cols].fillna(0) + s_row[cols].fillna(0)
                rows_to_drop_gunwi.append(s_idx)
                
    df.drop(list(set(rows_to_drop_gunwi)), inplace=True)
    rows_to_drop_gunwi = []

    # ---------------------------------------------------------
    # 2. 출장소 합산
    # ---------------------------------------------------------
    main_offices = ['죽장면', '낙동면', '모서면', '화북면', '은척면', '문경읍', '가은읍', '지품면', '축산면', '청도읍', '화양읍', '북면']
    for office in main_offices:
        mains = df[df['canon_name'] == office]
        for idx, main_row in mains.iterrows():
            code_prefix = str(main_row['canon_code'])[:4]
            mask = ((df['canon_name'].str.contains(office)) & (df['canon_name'].str.contains('출장소')) & (df['canon_code'].astype(str).str.startswith(code_prefix)))
            branches = df[mask]
            if not branches.empty:
                cols = [c for c in df.columns if c.startswith('Y')]
                for b_idx, branch_row in branches.iterrows():
                    df.loc[idx, cols] = df.loc[idx, cols].fillna(0) + branch_row[cols].fillna(0)
                rows_to_drop.extend(branches.index.tolist())
    df.drop(list(set(rows_to_drop)), inplace=True)
    rows_to_drop = []

    # ---------------------------------------------------------
    # 3. 특수 규칙 (코드 기반)
    # ---------------------------------------------------------
    # # 중앙동(4725047001) -> 동문동 (상주)
    # j_code = 4725047001
    # j_rows = df[df['canon_code'] == j_code]
    # if not j_rows.empty:
    #     idx = j_rows.index[0]
    #     prefix = str(df.loc[idx, 'canon_code'])[:4]
    #     target = df[(df['canon_name'] == '동문동') & (df['canon_code'].astype(str).str.startswith(prefix))]
    #     if not target.empty:
    #         t_idx = target.index[0]
    #         cols = [c for c in df.columns if c.startswith('Y')]
    #         df.loc[t_idx, cols] = df.loc[t_idx, cols].fillna(0) + df.loc[idx, cols].fillna(0)
    #         rows_to_drop.append(idx)
    # df.drop(list(set(rows_to_drop)), inplace=True)
    # rows_to_drop = []
    
    # ---------------------------------------------------------
    # 5. [추가] 영주동 1998년 변동 규칙 (1997년 데이터에 적용)
    # ---------------------------------------------------------
    # *영주1동 + 영주2동 + 영주3동 -> 영주1동 (Y1997 합산)
    # *영주4동 -> 영주2동 (Y1997 이동)
    
    yj_rows = df[df['canon_name'].str.contains('영주')]
    if not yj_rows.empty:
        prefix = str(yj_rows.iloc[0]['canon_code'])[:4]
        
        y1 = df[(df['canon_name'] == '영주1동') & (df['canon_code'].astype(str).str.startswith(prefix))]
        y2 = df[(df['canon_name'] == '영주2동') & (df['canon_code'].astype(str).str.startswith(prefix))]
        y3 = df[(df['canon_name'] == '영주3동') & (df['canon_code'].astype(str).str.startswith(prefix))]
        y4 = df[(df['canon_name'] == '영주4동') & (df['canon_code'].astype(str).str.startswith(prefix))]
        
        # 1997년 컬럼에만 적용
        target_cols = ['Y1997']
        valid_target_cols = [c for c in target_cols if c in df.columns]
        
        if valid_target_cols:
            if not y1.empty:
                y1_idx = y1.index[0]
                # Y1 = Y1 + Y2(Old) + Y3(Old)
                if not y2.empty:
                    s_idx = y2.index[0]
                    for col in valid_target_cols:
                        df.at[y1_idx, col] = (df.at[y1_idx, col] if pd.notna(df.at[y1_idx, col]) else 0) + (df.at[s_idx, col] if pd.notna(df.at[s_idx, col]) else 0)
                if not y3.empty:
                    s_idx = y3.index[0]
                    for col in valid_target_cols:
                        df.at[y1_idx, col] = (df.at[y1_idx, col] if pd.notna(df.at[y1_idx, col]) else 0) + (df.at[s_idx, col] if pd.notna(df.at[s_idx, col]) else 0)
                    rows_to_drop.append(s_idx) # Y3 삭제
            
            if not y2.empty and not y4.empty:
                y2_idx = y2.index[0]
                s_idx = y4.index[0]
                # Y2 = Y4(Old)
                for col in valid_target_cols:
                    df.at[y2_idx, col] = df.at[s_idx, col]
                rows_to_drop.append(s_idx) # Y4 삭제

    # ---------------------------------------------------------
    # 4. 일반 병합 규칙
    # ---------------------------------------------------------
    rules = [
        (['홍해읍'], '흥해읍'),
        (['용흥1동', '용흥2동'], '용흥동'),
        (['장양동'], '장량동'),
        (['대보면'], '호미곶면'),
        (['상대1동', '상대2동'], '상대동'),
        (['해도1동', '해도2동'], '해도동'),
        (['덕수동', '대흥동', '중앙동', '동빈동', '대신동', '학산동', '항구동'], '중앙동'),
        (['죽도1동', '죽도2동'], '죽도동'),
        (['성내동', '중앙동'], '중부동'),
        (['성동동', '황오동'], '황오동'),
        (['탑정동', '황남동'], '황남동'),
        (['인교동', '도동동', '보황동'], '월성동'),
        (['정래동', '불국동'], '불국동'),
        (['용호동', '모암동'], '용암동'),
        (['용암동'], '자산동'), 
        (['성내동', '남산동'], '성남동'),
        (['평화동', '성남동'], '평화남산동'),
        (['신음동', '금산동', '대응동'], '대신동'),
        (['미곡동', '부곡동'], '대곡동'),
        (['양천동', '양금동'], '양금동'),
        (['동구동', '중구동'], '중구동'),
        (['법상동', '서구동', '강남동'], '강남동'),
        (['용성동', '송천동'], '용상동'),
        (['산동면'], '산동읍'),
        (['원평2동', '원평3동'], '원평2동'), 
        (['원평1동', '원평2동'], '원평동'), 
        (['선주동', '원남동'], '선주원남동'),
        (['상모동', '사곡동'], '상모사곡동'),
        (['하망1동', '하망2동', '하망3동'], '하망동'),
        (['교동'], '서부동'),
        (['주남동', '봉작동', '영도동'], '남부동'),
        (['명산동', '대전동'], '중앙동'),
        (['중앙동', '동문동'], '동문동'),
        (['신기동', '대성동'], '신평동'),
        (['점촌동'], '점촌1동'),
        (['중앙동'], '점촌2동'),
        (['신흥동'], '점촌3동'),
        (['신평동'], '점촌4동'),
        (['모전동'], '점촌5동'),
        (['고로면'], '삼국유사면'), # 군위 외 다른 지역(혹은 잔여 데이터)을 위한 규칙 유지
        (['북삼면'], '북삼읍'),
        (['석적면'], '석적읍'),
        (['농소면','율곡동'], '율곡동'),
        (['서부1동', '서부2동', '서부동'], '서부1동')
    ]

    for srcs, dst in rules:
        potential_srcs = df[df['canon_name'].isin(srcs)]
        if potential_srcs.empty: continue
        prefixes = potential_srcs['canon_code'].astype(str).str[:4].unique()
        for prefix in prefixes:
            region_srcs = potential_srcs[potential_srcs['canon_code'].astype(str).str.startswith(prefix)]
            region_dst = df[(df['canon_name'] == dst) & (df['canon_code'].astype(str).str.startswith(prefix))]
            if not region_dst.empty:
                dst_idx = region_dst.index[0]
                cols = [c for c in df.columns if c.startswith('Y')]
                valid_srcs = region_srcs[region_srcs.index != dst_idx]
                for s_idx, s_row in valid_srcs.iterrows():
                    df.loc[dst_idx, cols] = df.loc[dst_idx, cols].fillna(0) + s_row[cols].fillna(0)
                rows_to_drop.extend(valid_srcs.index.tolist())
    df.drop(list(set(rows_to_drop)), inplace=True)

    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    return df

# 실행 예시 (파일명은 실제 경로에 맞게 수정)
final_df = merge_administrative_districts('data/emd_2024_check_1997_cc.csv', 'data/emd_2024_merged_1997_v3.csv')
print("Done. Rows:", len(final_df))

/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_1101/3481935504.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[idx, cols] = df.loc[idx, cols].fillna(0) + s_row[cols].fillna(0)
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_1101/3481935504.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[idx, cols] = df.loc[idx, cols].fillna(0) + s_row[cols].fillna(0)
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_1101/3481935504.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is d

Done. Rows: 321


/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_1101/3481935504.py:210: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[dst_idx, cols] = df.loc[dst_idx, cols].fillna(0) + s_row[cols].fillna(0)
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_1101/3481935504.py:210: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[dst_idx, cols] = df.loc[dst_idx, cols].fillna(0) + s_row[cols].fillna(0)
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_1101/3481935504.py:210: FutureWarning: Downcasting object dtype arrays on .fillna, 

### 행정구역코드 Join

In [ ]:
import pandas as pd
from pathlib import Path

# 0. 기본 경로 (필요하면 바꿔줘)
BASE = Path("/Users/kjh/Documents/innocity")  # or Path("/Users/kjh/Documents/...") 이런 식으로

# 1. 파일 로드
# 병합된 데이터 파일 (이전 단계 결과물)
merged_df = pd.read_csv(BASE / "data/emd_2024_merged_1997_v3.csv")

# 엑셀 원본 파일 경로
xlsx_file = BASE / "한국행정구역분류_2025.1.1.기준_20241231100033.xlsx"

# 시트 이름 (엑셀에서 보이는 탭 이름이랑 똑같이)
sheet_name = "2-2. 연계표_행정동 및 법정동(기준시점)"

# 헤더가 2번째 줄이면 header=1 (0-based index)
map_df = pd.read_excel(
    xlsx_file,
    sheet_name=sheet_name,
    header=1,          # 필요 없으면 0으로 바꾸면 됨
    engine="openpyxl", # 보통 이게 잘 됨
)

# 컬럼 이름 한번 확인해보기 (디버깅용)
print(map_df.columns)


Index(['시도', '시군구', '행정구역명', '행정동\n(행정기관명)', '법정동', '행정구역분류', '행정기관코드',
       '행정기관 생성일', '법정동코드', '법정동 관할구역\n분할여부', '행정동 영문명칭', '비고', 'Unnamed: 12',
       '내'],
      dtype='object')


/opt/miniconda3/envs/snustdatasci-2025/lib/python3.13/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [32]:
# 2. 매핑 테이블 전처리
# 필요한 컬럼만 선택: 행정기관코드(Join Key), 행정구역분류(Target Value)
mapping = map_df[["행정기관코드", "행정구역분류"]].copy()

# 중복 제거 (행정동-법정동 1:N 관계 등으로 인한 중복 행 제거)
mapping = mapping.drop_duplicates()

# 결측치 처리 (행정기관코드가 없는 행 제외)
mapping = mapping.dropna(subset=["행정기관코드"])

# 키 타입 통일 (int64) — canon_code도 같이 맞춰주기
mapping["행정기관코드"] = mapping["행정기관코드"].astype("int64")
merged_df["canon_code"] = merged_df["canon_code"].astype("int64")

# 3. Join 수행 (Left Join)
# canon_code와 행정기관코드를 기준으로 병합
final_df = merged_df.merge(
    mapping,
    left_on="canon_code",
    right_on="행정기관코드",
    how="left"
)

# 4. 컬럼 정리
# 행정구역분류 -> ADM_CD로 이름 변경
final_df = final_df.rename(columns={"행정구역분류": "ADM_CD"})

# 중복된 키 컬럼(행정기관코드) 삭제
final_df = final_df.drop(columns=["행정기관코드"])

# 5. 결과 확인 및 저장
print(final_df[["canon_code", "canon_name", "ADM_CD"]].head())

final_df.to_csv(
    BASE / "data/emd_2024_merged_final.csv",
    index=False,
    encoding="utf-8-sig"
)


   canon_code canon_name      ADM_CD
0  2772025000        군위읍  22520111.0
1  2772031000        소보면  22520311.0
2  2772032000        효령면  22520312.0
3  2772033000        부계면  22520313.0
4  2772034000        우보면  22520314.0
